In [1]:
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
folder = Path('/n/groups/datta/win/longtogeny/dlight')
files = sorted(folder.glob('**/results_00.h5'))

In [3]:
len(files)

174

In [4]:
str(files[0])

'/n/groups/datta/win/longtogeny/dlight/session_20230807113048-862691 (datta-realtime1)/proc/results_00.h5'

In [5]:
script = '''#!/bin/env bash
#SBATCH -c 3
#SBATCH -n 1
#SBATCH --mem=20G
#SBATCH -p short
#SBATCH -t 01:10:00
#SBATCH --output=/n/scratch/users/w/wg41/tmp/da-paper-ae-%j.out

source $HOME/.bashrc
conda activate rockwell-unet
python tmp.py "{file_path}"
'''

In [6]:
tmp_py = '''
import sys
import h5py
import numpy as np
import tensorflow as tf

file = sys.argv[1]

in_frames = 'win_size_norm_frames_v8'
out_frames = 'da_paper_frames'

ae = tf.keras.models.load_model('/n/groups/datta/win/reinforcement-models/realtime-packages/realtime-package-v6/autoencoder.h5')
ae.trainable = False

with h5py.File(file, 'a') as h5f:
    frames = h5f[in_frames][()]
    pred = ae.predict(frames[..., None]).squeeze()
    h5f.create_dataset(out_frames, data=pred)

print("DONE")
'''

with open("tmp.py", 'w') as f:
    f.write(tmp_py)

In [7]:
# %%bash

# source $HOME/.bashrc
# conda activate rockwell-unet
# python tmp.py '/n/groups/datta/win/longtogeny/dlight/session_20230807113048-862691 (datta-realtime1)/proc/results_00.h5'

In [8]:
for file in files:
    new_script = script.format(file_path=str(file))
    with open("tmp.sh", 'w') as f:
        f.write(new_script)
    !sbatch tmp.sh

Submitted batch job 32818997
Submitted batch job 32818999
Submitted batch job 32819000
Submitted batch job 32819001
Submitted batch job 32819002
Submitted batch job 32819003
Submitted batch job 32819004
Submitted batch job 32819005
Submitted batch job 32819006
Submitted batch job 32819007
Submitted batch job 32819008
Submitted batch job 32819009
Submitted batch job 32819010
Submitted batch job 32819011
Submitted batch job 32819012
Submitted batch job 32819013
Submitted batch job 32819014
Submitted batch job 32819015
Submitted batch job 32819016
Submitted batch job 32819017
Submitted batch job 32819018
Submitted batch job 32819019
Submitted batch job 32819020
Submitted batch job 32819021
Submitted batch job 32819022
Submitted batch job 32819023
Submitted batch job 32819024
Submitted batch job 32819025
Submitted batch job 32819026
Submitted batch job 32819027
Submitted batch job 32819028
Submitted batch job 32819029
Submitted batch job 32819030
Submitted batch job 32819031
Submitted batc

## Run PCA and apply whitening

In [4]:
import h5py
import joblib

In [5]:
with h5py.File('/n/groups/datta/win/reinforcement-models/realtime-packages/realtime-package-v6/pc_components.h5', 'r') as h5f:
    pc_components = h5f['components'][:10]

In [6]:
pc_components.shape

(10, 6400)

In [7]:
whitener = joblib.load('/n/groups/datta/win/reinforcement-models/realtime-packages/realtime-package-v6/preprocessor.p')

/home/wg41/miniconda3/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator FunctionTransformer from version 0.19.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [8]:
arhmm = joblib.load('/n/groups/datta/win/reinforcement-models/realtime-packages/realtime-package-v6/arhmm.p')

In [9]:
with h5py.File("/n/groups/datta/win/longtogeny/data/ontogeny/version_12/dlight_da_paper_syllables.h5", "w") as syllable_h5:
    for file in tqdm(files):
        try:
            with h5py.File(file, 'r') as h5f:
                uuid = h5f['metadata']['uuid'][()].decode()
                frames = h5f['da_paper_frames'][()]
            pcs = frames.reshape(len(frames), -1).dot(pc_components.T)
            pcs = whitener.transform(pcs)
            syllables = arhmm['model'].heldout_viterbi(pcs)
            syllable_h5.create_dataset(uuid, data=syllables)
        except KeyError:
            pass

  0%|          | 0/174 [00:00<?, ?it/s]